In [ ]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

import os
import tez            # a simple pytorch trainer
from tez.callbacks import EarlyStopping
import albumentations # Fast image augmentation library
import pandas as pd
import cv2            # OpenCV 
import numpy as np
import timm           # (Unofficial) PyTorch Image Models
import torch
import torch.nn as nn
from sklearn import metrics, model_selection
from tqdm import tqdm
import random


class Args:
    batch_size = 64
    image_size = 256
    epochs = 20 
    
path_input = r'../input/petfinder-pawpularity-score/'

test_aug = albumentations.Compose(
    [
        albumentations.Resize(Args.image_size, Args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [ ]:
class Dataset:
    '''To get data of each image'''
    def __init__(self, 
                 image_paths, 
                 dense_features, 
                 targets, 
                 augmentations=None):
        '''
        :param dense_features: chosen feature columns
            dense_features = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
                              'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
        '''
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        '''
        :param item: a row index of one image data
        return: dic
            Eg. dic = {'image': tensor data of the image,
                       'features': a dense features corresponding to this image, 
                       'targets': Pawpularity of this image}        
        '''        
        image = cv2.imread(self.image_paths[item]) # Eg. image.shape = (960, 720, 3): W, H and C
        
        # as opencv loads in BGR format by default, we want to show it in RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Converts an image from one color space to another
                                                       # image.shape = (960, 720, 3): C here order RGB
            
        if self.augmentations is not None: # refer to train_aug and valid_aug
            augmented = self.augmentations(image=image)
            image = augmented['image']
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32) # (0, 1, 2) -> (2, 0, 1)
                                                                  #  W, H, C   -> C, W, H  
        features = self.dense_features[item, :]  # item is row index. Ex. item=0 is the first image       
        targets = self.targets[item]  # value of Pawpularity of an image
        
        return {
            'image': torch.tensor(image, dtype=torch.float),  # np array to torch tensor
            'features': torch.tensor(features, dtype=torch.float), 
            'targets': torch.tensor(targets,  dtype=torch.float)            
        }        

In [ ]:
class Model(tez.Model):
    '''
    Note: in tez: Dataset === pytorch requires
                  model class: nn.Module (pytorch) -> tez.Model
    '''
    def __init__(self, model_name):
        '''
        :param model_name: name of model for timm.create_model()
            Eg1. model_name = 'tf_efficientnet_b0_ns'
            Eg2. model_name = 'swin_large_patch4_window12_384'
        '''
        super().__init__()
        
        # model: EfficientNet
        self.model = timm.create_model(model_name, 
                                       pretrained = False, 
                                       in_chans = 3)
        
        # Linear(in_features=1280, out_features=128, bias=True). print model to see the keywords
        # Inside this model: (classifier): Linear(in_features=1280, out_features=1000, bias=True)
        # change it to:      (classifier): Linear(in_features=1280, out_features=128, bias=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128) 
        
        # Dropout(p=0.1, inplace=False)
        self.dropout = nn.Dropout(0.1)
        
        # Linear(in_features=140, out_features=1, bias=True)
        self.out = nn.Linear(128 + 12, 1)  # 12 is from dense features -> 128+12=140
        
        self.step_scheduler_after = 'epoch'
        
        # Other ways
        # self.dropout = nn.Dropout(0.1)
        # self.dense1 = nn.Linear(140, 64)
        # self.dense2 = nn.Linear(64, 1)
        
        
    def forward(self, image, features, targets=None):
        x = self.model(image)  # x.shape = torch.Size([1, 128]). Eg. [[-0.0453,....0.1661]]
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.out(x)
        
        # Other ways
        # x = self.model(image)
        # x = self.dropout(x)
        # x = torch.cat([x, features], dim=1)
        # x = self.dense1(x)
        # x = torch.relu(x)
        # x = self.dense2(x)
        
        if targets is not None:
            loss = nn.MSELoss()(x, targets.view(-1, 1))
            metrics = self.monitor_metrics(x, targets)
            return x, loss, metrics
        return x, 0, {}  
        
        
    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        
        rmse = metrics.mean_squared_error(y_true=targets, 
                                          y_pred=outputs, 
                                          squared=False # True returns MSE, False returns RMSE
                                         )
        return {'rmse': rmse}
    
    
    def fetch_scheduler(self):
        '''
        For Learning rate
        Set the learning rate of each parameter group using a cosine annealing schedule
        '''
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer,
            T_0 = 10,
            T_mult = 1,
            eta_min = 1e-6,
            last_epoch = -1
        )
        return sch
    
    
    def fetch_optimizer(self):
        ''' Adam optimization algo'''
        opt = torch.optim.Adam(
            self.parameters(),
            lr = 1e-4
        )
        return opt

In [ ]:
def prediction(KFold=10):
    dense_features = [
            'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
        ]
    
    super_final_predictions = []
    for fold in range(KFold):   
        print(f'fold = {fold}')
        model = Model(model_name= 'tf_efficientnet_b0_ns')         
        # model.load(f"/kaggle/working/model_cv{fold}.bin", device="cuda", weights_only=True) 
        model.load(f"../input/model-cv01234/model_cv{fold}.bin", device="cuda", weights_only=True) 

        df_test = pd.read_csv(os.path.join(path_input, 'test.csv'))
        test_img_paths = [os.path.join(path_input, f'test/{img}.jpg') for img in df_test["Id"].values] 
        
        test_dataset = Dataset(
            image_paths=test_img_paths,
            dense_features=df_test[dense_features].values,
            targets=np.ones(len(test_img_paths)),
            augmentations=test_aug
        )
        
        test_predictions = model.predict(test_dataset, 
                                         batch_size=2*Args.batch_size, 
                                         n_jobs=-1)

        final_test_predictions = []
        for preds in tqdm(test_predictions):
            final_test_predictions.extend(preds.ravel().tolist())
        # End of for preds
        super_final_predictions.append(final_test_predictions)
    # End of for fold
    super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
    df_test["Pawpularity"] = super_final_predictions
    df_test = df_test[["Id", "Pawpularity"]]
    df_test.to_csv("submission.csv", index=False)
    return df_test

# RUN PREDICTION
df_test = prediction(KFold=5)